## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.2.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jitendra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3028, 4448, 2051, 1634, 2460, 528, 936, 4011, 3289, 4530],
 [2646, 1091, 271, 2107, 479, 1001, 142],
 [3242, 3475, 685, 2777],
 [1937, 1386, 4809, 3272, 925, 286],
 [1496, 479, 4592, 1819, 3886, 3355, 479, 58, 2977, 2635],
 [444,
  233,
  3705,
  2575,
  1754,
  3652,
  3115,
  3563,
  4786,
  2701,
  272,
  2382,
  537,
  2171,
  142],
 [1687, 270, 3940, 4615, 2639, 65, 881, 43, 2394, 3802, 2729],
 [380, 1769, 1233, 3068, 353, 3102, 3652, 3007, 2394, 3802, 2729],
 [1396, 1092, 3418, 4013, 4880, 3040, 841, 2716, 3652, 846],
 [4989, 1141, 1475, 3129, 3861, 4897, 2323, 471],
 [3307, 4681, 1639, 3277, 559, 4828, 1316, 503, 3648, 90, 1002],
 [3272, 3797, 2460, 3040, 3652, 353],
 [4007, 1503, 794, 1260, 2220, 3355, 4280, 88, 545],
 [2872, 3887, 1732, 4289, 4422, 3490, 1083, 2394, 3802, 2729],
 [1284, 798, 2310, 3497, 610, 2394, 3802, 2729],
 [2399, 4136, 3140, 2492, 3802, 1977, 4747, 78, 2440, 4878],
 [1261, 2585, 1091],
 [2778, 174, 4917, 3271, 3652, 111, 4260, 142],
 [4679, 3335, 271, 4

### Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4011 3289 4530]
 [   0    0    0 ...  479 1001  142]
 [   0    0    0 ... 3475  685 2777]
 ...
 [   0    0    0 ... 2394 3802 2729]
 [   0    0    0 ... 4334 1161  749]
 [   0    0    0 ... 4555 3447 4724]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3028,
       4448, 2051, 1634, 2460,  528,  936, 4011, 3289, 4530], dtype=int32)

In [45]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [29]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 13ms/step - loss: 0.2993 - accuracy: 0.8558 - val_loss: 0.2003 - val_accuracy: 0.9138
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1418 - accuracy: 0.9440 - val_loss: 0.2187 - val_accuracy: 0.9165
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0970 - accuracy: 0.9647 - val_loss: 0.2250 - val_accuracy: 0.9142
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0739 - accuracy: 0.9743 - val_loss: 0.3056 - val_accuracy: 0.9102
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0572 - accuracy: 0.9811 - val_loss: 0.3307 - val_accuracy: 0.9140
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0348 - accuracy: 0.9896 - val_loss: 0.3963 - val_accuracy: 0.9085
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0195 - accuracy: 0.9940 - val_loss: 0.4831 - val_accuracy: 0.9007

### Performance Metrics And Accuracy

In [30]:

y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test,y_pred1)

array([[3111,  308],
       [ 235, 2381]])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9100248550124275

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

